In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from PIL import Image
import os

In [ ]:
num_classes = 58
batch_size = 32
epochs = 10
learning_rate = 0.001

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.png')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, img_path


data_dir = '/kaggle/input/traffic-sign-dataset-classification/traffic_Data'
image_datasets = {
    'train': datasets.ImageFolder(
        root=f"{data_dir}/DATA",
        transform=data_transforms['train']
    ),
    'val': TestDataset(
        root_dir=f"{data_dir}/TEST",
        transform=data_transforms['val']
    )
}

In [ ]:
print("Class to Index Mapping:", image_datasets['train'].class_to_idx)

Class to Index Mapping: {'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22, '3': 23, '30': 24, '31': 25, '32': 26, '33': 27, '34': 28, '35': 29, '36': 30, '37': 31, '38': 32, '39': 33, '4': 34, '40': 35, '41': 36, '42': 37, '43': 38, '44': 39, '45': 40, '46': 41, '47': 42, '48': 43, '49': 44, '5': 45, '50': 46, '51': 47, '52': 48, '53': 49, '54': 50, '55': 51, '56': 52, '57': 53, '6': 54, '7': 55, '8': 56, '9': 57}


In [7]:
dataloaders = {x: DataLoader(
    image_datasets[x], batch_size=batch_size, shuffle=(x == 'train'), num_workers=4
) for x in ['train', 'val']}

if 'train' in image_datasets:
    dataset_sizes = {'train': len(image_datasets['train'])}
else:
    dataset_sizes = {}

In [ ]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 185MB/s] 


In [ ]:
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)


for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    print("-" * 10)

    for phase in ['train']:
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            if labels.max() >= num_classes:
                print(f"Invalid label detected: {labels.max().item()} (max allowed: {num_classes - 1})")
                continue

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

print("Training complete.")


Epoch 1/10
----------
train Loss: 1.9333 Acc: 0.5513
Epoch 2/10
----------
train Loss: 0.8939 Acc: 0.7801
Epoch 3/10
----------
train Loss: 0.6422 Acc: 0.8319
Epoch 4/10
----------
train Loss: 0.5573 Acc: 0.8504
Epoch 5/10
----------
train Loss: 0.4874 Acc: 0.8691
Epoch 6/10
----------
train Loss: 0.4273 Acc: 0.8801
Epoch 7/10
----------
train Loss: 0.4008 Acc: 0.8856
Epoch 8/10
----------
train Loss: 0.3826 Acc: 0.8918
Epoch 9/10
----------
train Loss: 0.3380 Acc: 0.9048
Epoch 10/10
----------
train Loss: 0.3258 Acc: 0.9089
Training complete.
